In [2]:
# ===================================================================
#  Library
# ===================================================================
import os
import random
import numpy as np
import pandas as pd
import polars as pl
from tqdm.auto import tqdm

from sklearn.metrics import mean_absolute_percentage_error
import warnings
warnings.simplefilter("ignore")

from scipy.optimize import minimize

In [3]:
# ===================================================================
#  CFG
# ===================================================================
class CFG:
    seed = 42
    save_dir = "G:/マイドライブ/signate_StudentCup2023/exp/"
    data_dir = "G:/マイドライブ/signate_StudentCup2023/data/"
    filename = "exp053"

In [4]:
# ===================================================================
#  Data Loading
# ===================================================================
df = pl.read_csv(CFG.save_dir+f"oof_df_exp050.csv").sort("id")
train = pl.read_csv(CFG.data_dir+"train.csv", columns=["id", "price"])
df = df.join(train, on="id", how="left")

test = pl.read_csv(CFG.save_dir+f"exp050.csv").sort("id")

df.head()

id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,pred_10,pred_11,pred_12,pred_13,pred_14,pred_15,pred_16,pred_17,pred_18,pred_19,pred_20,pred_21,pred_22,pred_23,pred_24,pred_25,pred_26,pred_27,pred_28,pred_29,price
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
0,6881.892301,6517.414858,7476.567746,7052.61059,6809.002174,6465.089852,7714.185588,7010.85559,7184.306903,6364.997182,7501.188408,7678.577536,8374.386963,8105.340176,8644.322072,7985.056344,8628.261509,6528.148787,7798.699012,8063.395359,8981.008775,8613.396141,10717.468387,9447.344769,7984.506685,8272.276158,5876.6578,6016.32209,7374.013382,8496.804984,27587
1,3740.634027,3496.012582,3789.907986,3322.844722,3603.935425,3485.623398,3507.497452,3714.057283,3488.353022,3750.770625,3412.19359,3927.363047,3886.662697,3551.474485,3713.3473,3463.093834,3695.630189,3663.237361,3463.024267,3905.364004,3563.378845,4112.248835,3360.300264,3807.072052,3406.217022,3615.842705,4245.317355,4453.437876,4408.398403,3900.820201,4724
2,2954.247573,2735.110086,3288.647709,2893.84021,3253.350282,2885.690295,3202.10377,2799.365349,2989.931642,2920.912332,3059.180374,3198.706345,2764.92988,3237.239548,2756.230854,3233.761663,2927.997318,3258.873698,3235.904715,3139.762054,3016.016944,3167.730032,2929.953707,2957.126627,3618.763534,3022.052454,3913.586712,3705.560742,2921.102463,3116.785537,10931
3,8430.949224,8337.416095,9033.353098,8081.40405,8187.453186,8477.54243,8546.771462,8388.923127,8119.18331,8413.930266,8298.903056,8142.383515,8055.886032,8356.561411,8421.656285,8167.994718,8646.470884,8423.019535,8016.7337,8041.633697,8331.325796,7947.686753,8057.31244,8250.578545,8052.200549,8115.355025,8157.420044,7434.588221,8284.727211,7855.624335,16553
4,3972.418866,4254.790314,4089.549517,4272.046434,4446.949951,4022.828501,4374.413436,4369.679027,4150.7095,3941.880817,4227.904345,4420.621374,4676.941221,4534.857719,4604.858395,4588.033874,4023.362549,4455.22004,4574.171104,4520.151023,4408.383964,4200.673767,4535.020561,4420.531149,4421.192901,4254.960815,4031.139467,3972.143434,4027.789862,4720.410259,5158


In [5]:
# ===================================================================
#  Utils
# ===================================================================
def seed_everything(seed):
    """fix random factors"""
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(CFG.seed)
    

def get_score(y_true, y_pred):
    """get MAPE score"""
    score = mean_absolute_percentage_error(y_true, y_pred)
    return score * 100

In [ ]:
def obj_func(x, *oof_list):
    oof = 0
    x[x<0] = 0
    x = x/np.sum(x)
    for i in range(min(len(x), len(oof_list))):
        oof += x[i]*oof_list[i][['pred_0', 'pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']]
    return get_score(oof.values, ground_truth.values)